In [3]:
import sys
!{sys.executable} -m pip install pandas
!{sys.executable} -m pip install backtesting



     |████████████████████████████████| 174 kB 2.0 MB/s eta 0:00:01
     |████████████████████████████████| 18.5 MB 3.8 MB/s eta 0:00:01
     |████████████████████████████████| 197 kB 7.2 MB/s eta 0:00:01
     |████████████████████████████████| 3.0 MB 8.3 MB/s eta 0:00:01
  Created wheel for backtesting: filename=Backtesting-0.3.2-py3-none-any.whl size=173652 sha256=24121b5d0edefda7a3c89d9122e7af94d56b37b0d65fc74b7615a3896765de11
  Stored in directory: /Users/tanvirsethi/Library/Caches/pip/wheels/0f/a3/58/9c8a45dd3c32e1b0f4637ee8828891fba11b7045867f3ff27b
Successfully built backtesting


In [4]:
import pandas as pd
df = pd.read_csv (r'/Users/tanvirsethi/Desktop/Klahhane Capital/Research on VIX/^VIX.csv')
print(df)

            Date       Open       High    Low      Close  Adj Close  Volume
0     2006-01-03  12.250000  12.510000  10.99  11.140000  11.140000       0
1     2006-01-04  11.220000  11.710000  10.97  11.370000  11.370000       0
2     2006-01-05  11.430000  11.840000  11.31  11.310000  11.310000       0
3     2006-01-06  11.230000  11.500000  10.81  11.000000  11.000000       0
4     2006-01-09  11.350000  11.350000  10.98  11.130000  11.130000       0
...          ...        ...        ...    ...        ...        ...     ...
3985  2021-11-01  16.850000  17.700001  16.32  16.410000  16.410000       0
3986  2021-11-02  16.540001  16.650000  15.89  16.030001  16.030001       0
3987  2021-11-03  16.110001  16.389999  14.90  15.100000  15.100000       0
3988  2021-11-04  15.060000  16.139999  14.73  15.440000  15.440000       0
3989  2021-11-05  15.590000  17.020000  14.95  16.480000  16.480000       0

[3990 rows x 7 columns]


In [12]:
%matplotlib inline
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import datetime
from tabulate import tabulate
import warnings
warnings.filterwarnings('ignore')
import pandas_datareader.data as web

def MovingAverageCrossStrategy(stock_symbol = '^VIX', start_date = '2006-01-01', end_date = '2021-11-06', 
                               short_window = 20, long_window = 50, moving_avg = 'SMA', display_table = True):
    '''
    The function takes the stock symbol, time-duration of analysis, 
    look-back periods and the moving-average type(SMA or EMA) as input 
    and returns the respective MA Crossover chart along with the buy/sell signals for the given period.
    '''
    # stock_symbol - (str)stock ticker as on Yahoo finance. Eg: 'ULTRACEMCO.NS' 
    # start_date - (str)start analysis from this date (format: 'YYYY-MM-DD') Eg: '2018-01-01'
    # end_date - (str)end analysis on this date (format: 'YYYY-MM-DD') Eg: '2020-01-01'
    # short_window - (int)lookback period for short-term moving average. Eg: 5, 10, 20 
    # long_window - (int)lookback period for long-term moving average. Eg: 50, 100, 200
    # moving_avg - (str)the type of moving average to use ('SMA' or 'EMA')
    # display_table - (bool)whether to display the date and price table at buy/sell positions(True/False)

    # import the closing price data of the stock for the aforementioned period of time in Pandas dataframe
    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-'))) 
    stock_df = web.DataReader(stock_symbol, 'yahoo', start = start, end = end)['Close']
    stock_df = pd.DataFrame(stock_df) # convert Series object to dataframe 
    stock_df.columns = {'Close Price'} # assign new colun name
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows 
                        
    # column names for long and short moving average columns
    short_window_col = str(short_window) + '_' + moving_avg
    long_window_col = str(long_window) + '_' + moving_avg  
  
    if moving_avg == 'SMA':
        # Create a short simple moving average column
        stock_df[short_window_col] = stock_df['Close Price'].rolling(window = short_window, min_periods = 1).mean()

        # Create a long simple moving average column
        stock_df[long_window_col] = stock_df['Close Price'].rolling(window = long_window, min_periods = 1).mean()

    elif moving_avg == 'EMA':
        # Create short exponential moving average column
        stock_df[short_window_col] = stock_df['Close Price'].ewm(span = short_window, adjust = False).mean()

        # Create a long exponential moving average column
        stock_df[long_window_col] = stock_df['Close Price'].ewm(span = long_window, adjust = False).mean()

    # create a new column 'Signal' such that if faster moving average is greater than slower moving average 
    # then set Signal as 1 else 0.
    stock_df['Signal'] = 0.0  
    stock_df['Signal'] = np.where(stock_df[short_window_col] > stock_df[long_window_col], 1.0, 0.0) 

    # create a new column 'Position' which is a day-to-day difference of the 'Signal' column. 
    stock_df['Position'] = stock_df['Signal'].diff()

    # plot close price, short-term and long-term moving averages
    plt.figure(figsize = (20,10))
    plt.tick_params(axis = 'both', labelsize = 14)
    stock_df['Close Price'].plot(color = 'k', lw = 1, label = 'Close Price')  
    stock_df[short_window_col].plot(color = 'b', lw = 1, label = short_window_col)
    stock_df[long_window_col].plot(color = 'g', lw = 1, label = long_window_col) 

    # plot 'buy' signals
    plt.plot(stock_df[stock_df['Position'] == 1].index, 
            stock_df[short_window_col][stock_df['Position'] == 1], 
            '^', markersize = 15, color = 'g', alpha = 0.7, label = 'buy')

    # plot 'sell' signals
    plt.plot(stock_df[stock_df['Position'] == -1].index, 
            stock_df[short_window_col][stock_df['Position'] == -1], 
            'v', markersize = 15, color = 'r', alpha = 0.7, label = 'sell')
    plt.ylabel('Price in ₹', fontsize = 16 )
    plt.xlabel('Date', fontsize = 16 )
    plt.title(str(stock_symbol) + ' - ' + str(moving_avg) + ' Crossover', fontsize = 20)
    plt.legend()
    plt.grid()
    plt.show()
    
    if display_table == True:
        df_pos = stock_df[(stock_df['Position'] == 1) | (stock_df['Position'] == -1)]
        df_pos['Position'] = df_pos['Position'].apply(lambda x: 'Buy' if x == 1 else 'Sell')
        print(tabulate(df_pos, headers = 'keys', tablefmt = 'psql'))

ModuleNotFoundError: No module named 'matplotlib'

In [14]:
import sys
!{sys.executable} -m pip install pandas
!{sys.executable} -m pip install backtesting
!{sys.executable} -m pip install matplotlib
!{sys.executable} -m pip install tabulate
!{sys.executable} -m pip install pandas_datareader
import pandas as pd
df = pd.read_csv (r'/Users/tanvirsethi/Desktop/Klahhane Capital/Research on VIX/^VIX.csv')
print(df)
import matplotlib
from matplotlib import pyplot as plt 
import numpy as np 
import matplotlib.pyplot as plt
import datetime
from tabulate import tabulate
import warnings
warnings.filterwarnings('ignore')
import pandas_datareader.data as web

long_leg_rel_strike = df['Adj Close'] / df['Low']
print(long_leg_rel_strike)
short_leg_rel_strike = 2.0

long_leg_days_to_exp = round(45.00)
short_leg_days_to_exp = round(20.00)

if long_leg_rel_strike >= short_leg_rel_strike & long_leg_days_to_exp >= short_leg_days_to_exp:
    def MovingAverageCrossStrategy(stock_symbol = '^VIX', start_date = '2006-01-01', end_date = '2021-11-06', 
                               short_leg_days_to_exp, long_leg_days_to_exp, moving_avg = 'SMA', display_table = True):
    
    


            Date       Open       High    Low      Close  Adj Close  Volume
0     2006-01-03  12.250000  12.510000  10.99  11.140000  11.140000       0
1     2006-01-04  11.220000  11.710000  10.97  11.370000  11.370000       0
2     2006-01-05  11.430000  11.840000  11.31  11.310000  11.310000       0
3     2006-01-06  11.230000  11.500000  10.81  11.000000  11.000000       0
4     2006-01-09  11.350000  11.350000  10.98  11.130000  11.130000       0
...          ...        ...        ...    ...        ...        ...     ...
3985  2021-11-01  16.850000  17.700001  16.32  16.410000  16.410000       0
3986  2021-11-02  16.540001  16.650000  15.89  16.030001  16.030001       0
3987  2021-11-03  16.110001  16.389999  14.90  15.100000  15.100000       0
3988  2021-11-04  15.060000  16.139999  14.73  15.440000  15.440000       0
3989  2021-11-05  15.590000  17.020000  14.95  16.480000  16.480000       0

[3990 rows x 7 columns]
0       1.013649
1       1.036463
2       1.000000
3       1.01